In [ ]:
!pip install transformers flask flask-cors pymupdf nltk twilio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.0 MB/s eta 0:00:00


In [2]:
import json
import fitz  # PyMuPDF for PDF processing
import nltk
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import pipeline
from twilio.rest import Client
import random


In [ ]:
# Load pre-trained transformer models for chatbot and summarization
chatbot = pipeline("text2text-generation", model="t5-small")
summarizer = pipeline("summarization")
nltk.download("punkt")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [4]:
def chatbot_response(user_input):
    """Generates a response using the NLP model"""
    response = chatbot(user_input, max_length=50, truncation=True)
    return response[0]["generated_text"]


In [5]:
def extract_text_from_pdf(pdf_path):
    """Extracts text from an uploaded PDF"""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

def summarize_text(text):
    """Summarizes extracted text using an NLP model"""
    return summarizer(text, max_length=150, min_length=50, do_sample=False)[0]["summary_text"]


In [6]:
bad_words = ["badword1", "badword2", "offensive_word"]  # Add actual words

def filter_inappropriate_language(text):
    """Replaces inappropriate words with ***"""
    words = text.split()
    filtered_text = " ".join("***" if word.lower() in bad_words else word for word in words)
    return filtered_text


In [7]:
# Twilio API Credentials (Replace with your Twilio credentials)
TWILIO_ACCOUNT_SID = "your_account_sid"
TWILIO_AUTH_TOKEN = "your_auth_token"
TWILIO_PHONE_NUMBER = "+your_twilio_number"

client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

def send_otp(phone_number):
    """Generates and sends OTP via Twilio"""
    otp = random.randint(100000, 999999)
    message = client.messages.create(
        body=f"Your verification code is {otp}",
        from_=TWILIO_PHONE_NUMBER,
        to=phone_number
    )
    return otp


In [8]:
app = Flask(__name__)
CORS(app)

@app.route("/chatbot", methods=["POST"])
def chatbot_api():
    data = request.json
    user_input = data.get("message", "")
    response = chatbot_response(user_input)
    filtered_response = filter_inappropriate_language(response)
    return jsonify({"response": filtered_response})

@app.route("/upload_pdf", methods=["POST"])
def upload_pdf():
    file = request.files["file"]
    text = extract_text_from_pdf(file)
    summary = summarize_text(text)
    return jsonify({"summary": summary})

@app.route("/send_otp", methods=["POST"])
def otp_verification():
    data = request.json
    phone_number = data.get("phone_number")
    otp = send_otp(phone_number)
    return jsonify({"otp": otp})


In [9]:
if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
